# TP1

## 1

In [1]:
import gym
import numpy as np
import pygame

## 2.1

In [2]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
env.render

<bound method Wrapper.render of <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>>

In [3]:
print(env.observation_space)

Discrete(16)


In [4]:
print(env.action_space)

Discrete(4)


In [5]:
print(env.P[0][2])

[(0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)]


In [6]:
print(env.P[3][1])

[(0.3333333333333333, 2, 0.0, False), (0.3333333333333333, 7, 0.0, True), (0.3333333333333333, 3, 0.0, False)]


In [7]:
env.reset()
env.step(1)

c:\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


(0, 0.0, False, False, {'prob': 0.3333333333333333})

In [8]:
random_action = env.action_space.sample()

In [9]:

env.reset()

(0, {'prob': 1})

## 2.2

In [10]:
def deroulement(num_timesteps:int, iteration:int=0):
    env.reset()
    i = 0
    random_action = env.action_space.sample()
    new_state, reward, done, info, prob = env.step(random_action)
    print(new_state, reward, done, info, prob)
    if done != True:
        while i < num_timesteps and done != True:
            random_action = env.action_space.sample()
            new_state, reward, done, info, prob = env.step(random_action)
            print(new_state, reward, done, info, prob)
            i += 1
        print(iteration)
    if reward != 1:       
        iteration+=1       
        deroulement(num_timesteps,iteration)
deroulement(20)
env.close()

0 0.0 False False {'prob': 0.3333333333333333}
0 0.0 False False {'prob': 0.3333333333333333}
0 0.0 False False {'prob': 0.3333333333333333}
1 0.0 False False {'prob': 0.3333333333333333}
0 0.0 False False {'prob': 0.3333333333333333}
4 0.0 False False {'prob': 0.3333333333333333}
0 0.0 False False {'prob': 0.3333333333333333}
4 0.0 False False {'prob': 0.3333333333333333}
8 0.0 False False {'prob': 0.3333333333333333}
4 0.0 False False {'prob': 0.3333333333333333}
0 0.0 False False {'prob': 0.3333333333333333}
1 0.0 False False {'prob': 0.3333333333333333}
2 0.0 False False {'prob': 0.3333333333333333}
6 0.0 False False {'prob': 0.3333333333333333}
5 0.0 True False {'prob': 0.3333333333333333}
0
4 0.0 False False {'prob': 0.3333333333333333}
4 0.0 False False {'prob': 0.3333333333333333}
4 0.0 False False {'prob': 0.3333333333333333}
8 0.0 False False {'prob': 0.3333333333333333}
9 0.0 False False {'prob': 0.3333333333333333}
8 0.0 False False {'prob': 0.3333333333333333}
9 0.0 False 

In [11]:
value_table = np.zeros(env.observation_space.n)
print(value_table)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# TP2
## 1

In [12]:
def value_iteration(env, gamma=1, threshold=1e-20, max_iterations=1000):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    V = np.zeros(n_states)
    for _ in range(max_iterations):
        delta = 0
        for s in range(n_states):
            v = V[s]
            action_values = [sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[s][a]) for a in range(n_actions)]
            V[s] = max(action_values)
            
            delta = max(delta, abs(v - V[s]))
        if delta < threshold:
            break
    pi = np.argmax([[sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[s][a]) for a in range(n_actions)] for s in range(n_states)], axis=1)
    return V, pi

optimal_values, optimal_policy = value_iteration(env)

print("Valeurs optimales:")
print(optimal_values)
print("Politique optimale:")
print(optimal_policy)    

Valeurs optimales:
[0.82352941 0.82352941 0.82352941 0.82352941 0.82352941 0.
 0.52941176 0.         0.82352941 0.82352941 0.76470588 0.
 0.         0.88235294 0.94117647 0.        ]
Politique optimale:
[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]


## 2

In [13]:
def policy_iteration(env, gamma=1, threshold=1e-20):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    V = np.zeros(n_states)
    pi = np.random.choice(n_actions, size=n_states)

    while True:
        while True:
            delta = 0
            for s in range(n_states):
                v = V[s]
                action = pi[s]
                transitions = env.P[s][action]
                V[s] = sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in transitions)
                delta = max(delta, abs(v - V[s]))
            if delta < threshold:
                break
        policy_stable = True
        for s in range(n_states):
            old_action = pi[s]
            pi[s] = np.argmax([sum(prob * (reward + gamma * V[next_state]) for prob, next_state, reward, _ in env.P[s][a]) for a in range(n_actions)])
            if old_action != pi[s]:
                policy_stable = False
        if policy_stable:
            return V, pi

optimal_values, optimal_policy = policy_iteration(env)

print("Valeurs optimales:")
print(optimal_values)
print("Politique optimale:")
print(optimal_policy)

Valeurs optimales:
[0.82352941 0.82352941 0.82352941 0.82352941 0.82352941 0.
 0.52941176 0.         0.82352941 0.82352941 0.76470588 0.
 0.         0.88235294 0.94117647 0.        ]
Politique optimale:
[0 3 3 3 0 0 0 0 3 1 0 0 0 2 1 0]
